### BioGRID interaction retrieval

In [10]:
import requests, sys, json
import pandas as pd
import pickle
import time

In [2]:
### BioGRID access key
BIOGRID_ACCESS_KEY = 'a207ffa23db79e55a7d7a5a69b79f0c9'

### BioGRID server
biogrid_server = f'https://webservice.thebiogrid.org/interactions/?accesskey={BIOGRID_ACCESS_KEY}'

In [31]:
with open('../Results/GENIE3/SARS-CoV-2/interactions_dataframe_with_entrez.pickle', 'rb') as f:
    interactions = pickle.load(f)

interactions

,gene1,gene2,score,gene1_name,gene2_name,gene1_entrez,gene2_entrez
0,ENSG00000200336,ENSG00000201695,0.102868,5S_rRNA,5S_rRNA,106480763,106865375
1,ENSG00000201695,ENSG00000200336,0.085720,5S_rRNA,5S_rRNA,106865375,106480763
2,ENSG00000273590,ENSG00000205670,0.081955,LOC102723553,LOC102723553,102723553,54065
3,ENSG00000205670,ENSG00000273590,0.077385,LOC102723553,LOC102723553,54065,102723553
4,ENSG00000160201,ENSG00000275895,0.072684,LOC102724594,LOC102724594,7307,102724594
...,...,...,...,...,...,...,...
249495,ENSG00000137673,ENSG00000201965,0.000000,MMP7,Y_RNA,4316,None
249496,ENSG00000206918,ENSG00000199546,0.000000,U6,Y_RNA,106480085,None
249497,ENSG00000252714,ENSG00000240665,0.000000,5S_rRNA,LSP1P2,100873645,100873967
249498,ENSG00000228078,ENSG00000202354,0.000000,HLA-U,RNY3,None,6085


In [4]:
interactions = pd.read_csv("../Results/Undirected_Results/GENIE3_net_gene_symbols_undirected.tsv", sep="\t")


interactions

,TF,target,importance
0,HLA-DQB2,RPL13P12,0.061263
1,CD200R1L-AS1,LINC01806,0.060807
2,F8A3,THNSL2,0.058155
3,LDHAP4,TEKT4P2,0.057168
4,TTTY15,XIST,0.057020
...,...,...,...
62830,LDHAP4,RNY3,0.000003
62831,NLGN4Y,RNU6ATAC,0.000000
62832,RNU6ATAC,TTTY15,0.000000
62833,SNORA80A,TTTY14,0.000000


In [24]:
with open('../Data/SARS-CoV-2/ensembl_gene_dict.pickle', 'rb') as f:
    ensembl_dict = pickle.load(f)

with open('../Data/SARS-CoV-2/ensembl_entrez_dict.pickle', 'rb') as f:
    ensembl_entrez_dict = pickle.load(f)

In [26]:
name_to_ensembl_dict = {val : key for key, val in ensembl_dict.items()}
name_to_entrez = {name : ensembl_entrez_dict[ens_id] for name, ens_id in name_to_ensembl_dict.items()}
name_to_entrez

{'NOS2': '4843',
 'LTF': '4057',
 'KDM5D': '8284',
 'CYP24A1': '1591',
 'NLRP2': '55655',
 'MYOM2': '9172',
 'TNC': '3371',
 'PLEKHH1': '57475',
 'ST3GAL6': '10402',
 'DDX3Y': '8653',
 'ZFY': '7544',
 'PTGS2': '5743',
 'CA12': '771',
 'LAMP3': '27074',
 'CXCL2': '2920',
 'PHACTR3': '116154',
 'OAS1': '4938',
 'SLC26A4': '5172',
 'TF': '7018',
 'CCDC80': '151887',
 'ADA2': '51816',
 'PRKY': None,
 'GADD45B': '4616',
 'PRODH': '5625',
 'MMP9': '4318',
 'MSLN': '10232',
 'COTL1': '23406',
 'PTN': '5764',
 'RARRES2': '5919',
 'GIMAP2': '26157',
 'RIGI': '23586',
 'FOLR1': '2348',
 'SLC15A3': '51296',
 'OAS3': '4940',
 'OAS2': '4939',
 'VNN2': '8875',
 'HGD': '3081',
 'RBP1': '5947',
 'TTTY15': '8287',
 'STAT1': '6772',
 'ARTN': '9048',
 'MAN1C1': '57134',
 'MUC5B': '727897',
 'RARRES1': '5918',
 'IFIT3': '3437',
 'IFIT2': '3433',
 'CSTA': '1475',
 'PLAU': '5328',
 'CXCL6': '6372',
 'PAX8': '7849',
 'STATH': '6779',
 'APOBEC3A_B': '200315',
 'KRT17': '3872',
 'CD68': '968',
 'RNASE1': '6035

In [7]:
top_df = interactions.sort_values(by="importance", ascending=False).head(100)

In [8]:
# BioGRID API setup 
BIOGRID_ACCESS_KEY = "a207ffa23db79e55a7d7a5a69b79f0c9"
BIOGRID_URL = "https://webservice.thebiogrid.org/interactions/"

In [35]:
def query_biogrid(gene1, gene2):
    params = {
        "accesskey": BIOGRID_ACCESS_KEY,
        "format": "json",
        "geneList": f"{gene1}|{gene2}",
        "searchNames": "true",
        "searchIds" : "true",
        "includeInteractors": "false",
        "includeHeader": "true"
    }
    try:
        response = requests.get(BIOGRID_URL, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()
        return (len(data) > 0), data  # true if interaction found
    except requests.exceptions.Timeout:
        print(f"Timeout when querying {gene1}, {gene2}")
    except requests.exceptions.ConnectionError as e:
        print(f"Connection error when querying {gene1}, {gene2}: {e}")
    except Exception as e:
        print(f"Unexpected error for {gene1}, {gene2}: {e}")
    return False

In [39]:
gene1, gene2 = 'HLA-DQA1', 'HLA-DQB1'

params = {
    "accesskey": BIOGRID_ACCESS_KEY,
    "format": "json",
    "geneList": f"{gene1}|{gene2}",
    "searchNames": "true",
    "includeInteractors": "false",
    "includeHeader": "true"
}

response = requests.get(BIOGRID_URL, params=params, timeout=10)
response.raise_for_status()
data = response.json()

dataset = pd.DataFrame.from_dict(data, orient="index")

columns = [
    "INTERACTION_ID",
    "SYSTEMATIC_NAME_A",
    "SYSTEMATIC_NAME_B",
    "OFFICIAL_SYMBOL_A",
    "OFFICIAL_SYMBOL_B",
    "EXPERIMENTAL_SYSTEM",
    "PUBMED_ID",
    "PUBMED_AUTHOR",
    "THROUGHPUT",
    "QUALIFICATIONS",
]
# dataset = dataset[columns]
dataset

,BIOGRID_INTERACTION_ID,ENTREZ_GENE_A,ENTREZ_GENE_B,BIOGRID_ID_A,BIOGRID_ID_B,SYSTEMATIC_NAME_A,SYSTEMATIC_NAME_B,OFFICIAL_SYMBOL_A,OFFICIAL_SYMBOL_B,SYNONYMS_A,...,PUBMED_ID,ORGANISM_A,ORGANISM_B,THROUGHPUT,QUANTITATION,MODIFICATION,ONTOLOGY_TERMS,QUALIFICATIONS,TAGS,SOURCEDB
2598501,2598501,3117,3119,109362,109364,DAQB-109B10.1,DADB-249P12.2,HLA-DQA1,HLA-DQB1,CD|CELIAC1|DQ-A1|GSE|HLA-DQA,...,29997244,9606,9606,High Throughput,-,-,{},BRET,-,BIOGRID
2598524,2598524,3119,3117,109364,109362,DADB-249P12.2,DAQB-109B10.1,HLA-DQB1,HLA-DQA1,CELIAC1|HLA-DQB|IDDM1,...,29997244,9606,9606,High Throughput,-,-,{},-,-,BIOGRID
2598538,2598538,3119,3117,109364,109362,DADB-249P12.2,DAQB-109B10.1,HLA-DQB1,HLA-DQA1,CELIAC1|HLA-DQB|IDDM1,...,29997244,9606,9606,High Throughput,-,-,{},BRET,-,BIOGRID
2598559,2598559,3117,3119,109362,109364,DAQB-109B10.1,DADB-249P12.2,HLA-DQA1,HLA-DQB1,CD|CELIAC1|DQ-A1|GSE|HLA-DQA,...,29997244,9606,9606,High Throughput,-,-,{},-,-,BIOGRID
3192531,3192531,3117,3119,109362,109364,DAQB-109B10.1,DADB-249P12.2,HLA-DQA1,HLA-DQB1,CD|CELIAC1|DQ-A1|GSE|HLA-DQA,...,33961781,9606,9606,High Throughput,0.999847476,-,{},BioPlex HCT HCT116 cells CompPASS score = 0.99...,-,BIOGRID


In [37]:
# loop through top pairs + check BioGRID
verified_count = 0
for index, row in top_df.iterrows():
    gene1, gene2 = row['TF'], row['target']
    found, data = query_biogrid(gene1, gene2)
    if found:
        verified_count += 1
        print(gene1, gene2)
    else:
        print('not found')
    time.sleep(1)  # API rate limits

print(f"{verified_count}/100 interactions found in BioGRID")

not found
not found
not found
not found
not found
not found
not found
not found
not found
not found
not found
not found
not found
not found
not found
not found
not found
not found
not found
not found
not found
not found
not found
not found
not found
not found
not found
not found
not found
not found
DDIT3 GADD45B
not found
not found
not found
not found
not found
not found
not found
not found
CCN1 FOS
not found
ECM1 HOPX
not found
not found
HLA-DQA1 HLA-DQB1
not found
not found
not found
not found
not found
not found
not found
not found
not found
not found
not found
not found
not found
not found
not found
not found
not found
not found
not found
not found
not found
not found
CXCL10 CXCL11
not found
not found
not found
not found
not found
not found
not found
not found
not found
not found
not found
not found
OAS2 STAT1
HLA-H LSP1
not found
not found
not found
not found
not found
not found
KRT17 THBS1
not found
not found
not found
not found
CXCL2 CXCL8
not found
not found
not found
not found

In [14]:
top_ints = interactions[['gene1_entrez', 'gene2_entrez']].iloc[:100].values
top_ints

array([['106480763', '106865375'],
       ['106865375', '106480763'],
       ['102723553', '54065'],
       ['54065', '102723553'],
       ['7307', '102724594'],
       ['283955', None],
       ['100008587', '109910384'],
       ['109910381', '124907150'],
       ['100271063', None],
       ['3120', '388344'],
       ['100008587', '109864274'],
       [None, None],
       ['109864274', '109910381'],
       ['109864281', '124907150'],
       ['55258', '474384'],
       ['124907150', '109864281'],
       ['124907150', '106632260'],
       ['124907150', '124907150'],
       ['109864281', '109910384'],
       [None, None],
       ['109910384', '109910381'],
       ['106632260', '109910384'],
       ['7503', '8287'],
       ['109864274', '100008587'],
       [None, '283955'],
       ['105376805', None],
       [None, None],
       ['8363', '8362'],
       ['109910381', '100008587'],
       [None, '7544'],
       ['57834', '729654'],
       ['109864274', '124907150'],
       ['109910381', '1

In [16]:
gene1, gene2 = top_ints[1]
print(gene1, gene2)
# for gene1, gene2 in top_ints:
# the rules outlined in the Wiki: https://wiki.thebiogrid.org/doku.php/biogridrest
params = {
    "accesskey": BIOGRID_ACCESS_KEY,
    "format": "json",  # Return results in TAB2 format
    "geneList": "|".join(gene1),  # Must be | separated
    "searchNames": "true",  # Search against official names
    "includeInteractors": "false",  # Set to true to get any interaction involving EITHER gene, set to false to get interactions between genes
    "includeHeader": "true",
}

# Additional options to try, you can uncomment them as necessary
# params["start"] = 5 # Specify where to start fetching results from if > 10,000 results being returned
# params["max"] = 10 # Specify the number of results to return, max is 10,000
# params["interSpeciesExcluded"] = "false" # true or false, If ‘true’, interactions with interactors from different species will be excluded (ex. no Human -> Mouse interactions)
# params["selfInteractionsExcluded"] = "false" # true or false, If ‘true’, interactions with one interactor will be excluded. (ex. no STE11 -> STE11 interactions)
# params["searchIds"] = "true" # true or false, If ‘true’, ENTREZ_GENE, ORDERED LOCUS and SYSTEMATIC_NAME (orf) will be examined for a match with the geneList
# params["searchSynonyms"] = "false" # true or false, If ‘true’, SYNONYMS will be examined for a match with the geneList
# params["searchBiogridIds"] = "false" # true or false, If ‘true’, BIOGRID INTERNAL IDS will be examined for a match with the geneList
# params["excludeGenes"] = "false" # true or false, If 'true' the geneList becomes a list of genes to EXCLUDE rather than to INCLUDE
# params["includeInteractorInteractions"] = "true" # true or false, If ‘true’ interactions between the geneList’s first order interactors will be included. Ignored if includeInteractors is ‘false’ or if excludeGenes is set to ‘true’.
# params["htpThreshold"] = 50 # Any publication with more than this many interactions will be excluded
# params["throughputTag"] = "any" # any, low, high. If set to low, only `low throughput` interactions will be returned, if set to high, only `high throughput` interactions will be returned
# params["additionalIdentifierTypes"] = "SGD|FLYBASE|REFSEQ" # You can specify a | separated list of additional identifier types to search against (see get_identifier_types.py)

r = requests.get(biogrid_server, params=params)
interactions = r.json()

# Create a hash of results by interaction identifier
data = {}
for interaction_id, interaction in interactions.items():
    data[interaction_id] = interaction
    # Add the interaction ID to the interaction record, so we can reference it easier
    data[interaction_id]["INTERACTION_ID"] = interaction_id

# Load the data into a pandas dataframe
dataset = pd.DataFrame.from_dict(data, orient="index")

columns = [
    "INTERACTION_ID",
    "SYSTEMATIC_NAME_A",
    "SYSTEMATIC_NAME_B",
    "OFFICIAL_SYMBOL_A",
    "OFFICIAL_SYMBOL_B",
    "EXPERIMENTAL_SYSTEM",
    "PUBMED_ID",
    "PUBMED_AUTHOR",
    "THROUGHPUT",
    "QUALIFICATIONS",
]
# dataset = dataset[columns]
dataset


106865375 106480763


AttributeError: 'list' object has no attribute 'items'